In [22]:
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import argparse

In [13]:
batch_size =5
learning_rate = 0.0001
dropout_level = 0.05
nz = 64

In [14]:
# weights initliazer
def weights_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nninit.zeros_(m.bias)

In [15]:
class EEG_Generator(nn.Module):
    def __init__(self):
        super(EEG_CNN_Gen, self).__init__()

        self.nz = nz
        self.layer1 = nn.Sequential(
            nn.Linear(self.nz, 640),
            nn.PReLU()
        )
        self.layer2 = nn.Sequential(
            nn.ConvTranspose1d(in_channels = 16, out_channels = 16, kernel_size = 22, stride = 4),
            nn.PReLU()
        )
        self.layer3 = nn.Sequential(
            nn.ConvTranspose1d(in_channels = 16, out_channels = 16, kernel_size = 18, stride = 2), 
            nn.PRelu()
        )
        self.layer4 = nn.Sequential(
            nn.ConvTranspose1d(in_channels = 16, out_channels = 2, kernel_size = 16, stride = 4),
            nn.Sigmoid()
        )
    
    def forward(self, z):
        out = self.layer(z)
        out = out.view(out.size(0), 16, 40)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

In [16]:
class EEG_Discriminator(nn.Module):
    def __init__(self):
        super(EEG_Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=2, out_channels = 16, kernel_size = 10, stridde = 2),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(0.2), 
            nn.MaxPool1d(2)
        )

        self.dense_layers = nn.Sequential(
            nn.Linear(5968, 600), 
            nn.LeakyReLU(0.2), 
            nn.Linear(600, 1)
        )

        def forward(self, x):
            out = self.layer1(x)
            out = out.view(out.size(0), -1)
            out = self.dense_layers(out)
            return out
        

In [17]:
def dcgan(datarain, label, nseed):
    random.seed(nseed)
    np.random.seed(nseed)
    torch.manual_seed(nseed)
    
    datatrain = torch.from_numpy(datatrain)
    label = torch.from_numpy(label)

    dataset = torch.utils.data.TensorDataset(datatrain, label)
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

    gen = EEG_Generator().to(device)
    dis = EEG_Discriminator().to(device)
    gen.apply(weights_init)
    dis.apply(weights_init)

    # loss function
    adversarial_loss = nn.BCEWithLogitsLoss()

    # Opt
    optimizer_G = torch.optim.Adam(gen.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(dis.parameters(), lr=learning_rate, betas=(0.5, 0.999))

    Tensor = torch.cuda.Floattensor if torch.cuda.is_available else torch.FloatTensor

    real_label = 1
    fake_label = 0 
    new_data =[]
    global_step = 0

    # TRAINING GAN
    dis.train()
    gen.train()

    for epoch in range(1200):
        for i, data in enumerate(dataloader, 0):
            imgs, _ = data
            imgs = imgs.to(device)

            # configure inputs
            real_data = imgs.type(Tensor)
            label = torch.ones(imgs.shape[0], 1).to(device)
            z = torch.randn(imgs.shape[0], nz).to(device)

            if i % 5 == 0:
                # train discriminator

                print("Discriminator")

                ooptimizer_D.zero_grad()
                output_real = dis(real_data)
                
                # calclaute error and backprop
                errD_real = adversarial_loss(output_real, label)
                errD_real.backward()

                # train with fake
                fake_data = generator(z)
                label = torch.zeros(imgs.shape[0], 1).to(device)
                output_fake = dis(fake_data)
                errD_fake = adversarial_loss(output_fake, label)
                errD_fake.backward()
                errD = errD_real + errD_fake
                optimizer_D.step()

            if i % 1 == 0:
                # train generator
                print("Generator")

                z = torch.randn(imgs.shape[0], nz).to(device)
                fake_data = gen(z)

                # reset grads
                optimizer_G.zero_grad()

                output = dis(fake_data)
                bceloss = adversarial_loss(output, torch.ones(imgs.shape[0], 1).to(device))
                errG = bceloss
                errG.backward()
                optimizer_G.step()
            
            print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, 1200, i, len(dataloader), errD.item(), errG.item()))

    # geenrate data
    dis.eval()
    gen.eval()
    for epcoh in range(100):
        for i, data in enumerate(dataloader, 0):
            imgs, _ = data
            imgs = imgs.to(device)
            z = torch.randn(imgs.shape[0], nz).to(device)

            fake_data = gen(z)
            output = dis(fake_data)

            if i % 200 == 0:
                fake_data = fake_data.data[:25].cpu().numpy()
                new_data.append(fake_data)
    
    new_data = np.concatenate(new_data)
    new_data = np.asarray(new_data)
    print(new_data.shape)
    return new_data